In [1]:
import seaborn as sns
import os
import numpy as np
import pandas as pd
from glob import glob
import matplotlib.pyplot as plt
import scipy
import PIL 
from PIL import Image 
import sklearn 
from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from scipy.spatial.distance import pdist, squareform
from mpl_toolkits.mplot3d import Axes3D
import os
import cv2
import fcswrite
import re


In [ ]:
#Single FOV, all in one timepoint/folder (Standard export format)



def process_grayscale_images_by_well(folder_path):
    expected_channels = ["w1", "w2", "w3", "w4"]
    well_pattern = re.compile(r'([A-F][0-9]{2})')
    channel_pattern = re.compile(r'(w[1-4])')

    images_by_well = {}

    for filename in os.listdir(folder_path):
        if filename.startswith("._") or not filename.endswith(".TIF"):
            continue

        well_match = well_pattern.search(filename)
        channel_match = channel_pattern.search(filename)

        if well_match and channel_match:
            well = well_match.group(1)
            channel = channel_match.group(1)

            images_by_well.setdefault(well, {})[channel] = os.path.join(folder_path, filename)

    for well, channel_files in images_by_well.items():
        print(f"Processing well: {well}")
        data = {"X": [], "Y": []}
        all_positions = []
        
        for channel in expected_channels:
            if channel in channel_files:
                image = Image.open(channel_files[channel])
                image_array = np.array(image)

                # #BACKGROUND SUBTRACTION
                # percentile_subtract = np.percentile(image_array, 30)
                # image_array = np.clip(image_array - percentile_subtract, a_min=0, a_max=None)
                # #BACKGROUND SUBTRACTION

                height, width = image_array.shape

                if len(all_positions) == 0:
                    positions = np.indices((height, width)).reshape(2, -1).T
                    all_positions = positions
                    data["X"], data["Y"] = positions.T

                data[channel] = image_array.flatten()
            else:
                data[channel] = np.zeros(len(all_positions))

        df = pd.DataFrame(data)
        df["total_intensity"] = df[expected_channels].sum(axis=1)
       

        for channel in expected_channels:
            df[f"p_{channel}"] = np.where(df["total_intensity"] > 0,
                                          df[channel] / df["total_intensity"], 0)

        #Change to fit your nomenclature 
        output_fcs_path = os.path.join(folder_path, f"d6_02_{well}.fcs")
        parameter_names = ["X", "Y", "texas red", "w2", 'w3', 'w4', "total_intensity"]
        data_matrix = df[parameter_names].to_numpy(dtype=np.float64)
        fcswrite.write_fcs(output_fcs_path, parameter_names, data_matrix)

        #Use below to export to CSV to troubleshoot
        # df.to_csv(output_csv_path, index=False)

        print(f"Saved FCS for well {well} to {output_fcs_path}")

# Run on the full image folder, change 
image_folder = "/Volumes/Trevor/Melina 2D3D/MX Format/Melina pilot 2D pool d6 02_Plate_52522/TimePoint_1"
process_grayscale_images_by_well(image_folder)


In [ ]:
#Folder of folders


def process_all_plates(root_folder):
    expected_channels = ["w1", "w2"]
    well_pattern = re.compile(r'([A-F][0-9]{2})')
    channel_pattern = re.compile(r'(w[1-4])')

    output_base = "/Volumes/Elements/Dox NH2 July 2025/Per Well Median Background Subtraction"
    os.makedirs(output_base, exist_ok=True)

    for subfolder in os.listdir(root_folder):
        subfolder_path = os.path.join(root_folder, subfolder)
        timepoint_path = os.path.join(subfolder_path, "TimePoint_1")

        if not os.path.isdir(timepoint_path):
            continue

        # Extract prefix for file naming (e.g., d0p1)
        prefix_match = re.search(r'D(\d+)\s+P(\d+)', subfolder, re.IGNORECASE)
        if not prefix_match:
            print(f"Skipping {subfolder}, couldn't parse day/plate info.")
            continue
        prefix = f"d{prefix_match.group(1).lower()}p{prefix_match.group(2).lower()}"

        images_by_well = {}
        for filename in os.listdir(timepoint_path):
            if filename.startswith("._") or not filename.endswith(".TIF"):
                continue

            well_match = well_pattern.search(filename)
            channel_match = channel_pattern.search(filename)

            if well_match and channel_match:
                well = well_match.group(1)
                channel = channel_match.group(1)
                images_by_well.setdefault(well, {})[channel] = os.path.join(timepoint_path, filename)

        for well, channel_files in images_by_well.items():
            print(f"Processing well: {well} from {subfolder}")
            data = {"X": [], "Y": []}
            all_positions = []

            for channel in expected_channels:
                if channel in channel_files:
                    image = Image.open(channel_files[channel])
                    image_array = np.array(image)

                    #Background Subtraction
                    percentile_subtract = np.percentile(image_array, 50)
                    image_array = np.clip(image_array - percentile_subtract, a_min=0, a_max=None)
                    #Background Subtraction
            
                    height, width = image_array.shape

                    if len(all_positions) == 0:
                        positions = np.indices((height, width)).reshape(2, -1).T
                        all_positions = positions
                        data["X"], data["Y"] = positions.T

                    data[channel] = image_array.flatten()
                else:
                    data[channel] = np.zeros(len(all_positions))

            df = pd.DataFrame(data)
            df["total_intensity"] = df[expected_channels].sum(axis=1)
            df["r_y"] = df['w1']/df["w2"]
            df["r_g"] = df['w1']/df["w3"]
            df["r_b"] = df['w1']/df["w4"]
            df["y_g"] = df['w2']/df["w3"]
            df["y_b"] = df['w2']/df["w4"]
            df["g_b"] = df['w3']/df["w4"]



            for channel in expected_channels:
                df[f"p_{channel}"] = np.where(df["total_intensity"] > 0,
                                              df[channel] / df["total_intensity"], 0)

            parameter_names = ["X", "Y", "texas_red", "yfp", "fitc", "cfp", "total_intensity", "r_y", "r_g", "r_b", "y_g", "y_b", "g_b"]
            data_matrix = df[["X", "Y", "w1", "w2", "w3", "w4", "total_intensity", "r_y", "r_g", "r_b", "y_g", "y_b", "g_b"]].to_numpy(dtype=np.float64)

            output_fcs_path = os.path.join(output_base, f"{prefix}_{well}.fcs")
            write_fcs(output_fcs_path, parameter_names, data_matrix)
            print(f"Saved: {output_fcs_path}")

# Usage
root_folder = "/Volumes/Elements/Dox NH2 July 2025/Raw Data"
process_all_plates(root_folder)
